<center><a href="https://sites.google.com/fat.uerj.br/livia/"> <img src="../images/capa2.png" alt="Header" style="width: 800px;"/> </a></center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vnikoofard/DeepLearningTF/blob/main/notebooks/03_asl_cnn.ipynb)

# Redes Neurais Convolucionais (*Convolutional Neural Networks* - CNNs)

Na seção anterior, construímos e treinamos um modelo simples para classificar imagens ASL. O modelo foi capaz de aprender como classificar corretamente o conjunto de dados de treinamento com precisão muito alta, mas não teve um desempenho tão bom no conjunto de dados de validação. Esse comportamento de não generalizar bem para dados que não são de treinamento é chamado de [overfitting](https://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html) e, nesta seção, apresentaremos um tipo popular de modelo chamado [rede neural convolucional](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53) que é especialmente bom para ler imagens e classificá-las.

## Objetivos

* Prepare dados especificamente para uma CNN
* Crie um modelo CNN mais sofisticado, compreendendo uma maior variedade de camadas do modelo
* Treine um modelo CNN e observe seu desempenho

## Loading and Preparing the Data

The below cell contains the data preprocessing techniques we learned in the previous labs. Review it and execute it before moving on:

In [ ]:
import tensorflow.keras as keras
import pandas as pd

# Load in our data from CSV files
train_url = "https://github.com/vnikoofard/DeepLearningTF/raw/main/data/asl_data/sign_mnist_train.csv"
valid_url = "https://github.com/vnikoofard/DeepLearningTF/raw/main/data/asl_data/sign_mnist_valid.csv"

train_df = pd.read_csv(train_url)
valid_df = pd.read_csv(valid_url)

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

## Reformatando imagens para uma CNN

No último exercício, as imagens individuais em nosso conjunto de dados estão no formato de longas listas de 784 pixels:

In [ ]:
x_train.shape, x_valid.shape

Nesse formato, não temos todas as informações sobre quais pixels estão próximos uns dos outros. Por causa disso, não podemos aplicar convoluções que detectam feições. Vamos reformatar nosso conjunto de dados para que eles estejam em um formato de 28x28 pixels. Isso permitirá que nossas convoluções associem grupos de pixels e detectem características importantes.

Observe que para a primeira camada convolucional do nosso modelo, precisamos ter não apenas a altura e a largura da imagem, mas também o número de [canais de cores](https://www.photoshopessentials.com/essentials/rgb/) . Nossas imagens são em tons de cinza, então teremos apenas 1 canal.

Isso significa que precisamos converter a forma atual `(27455, 784)` para `(27455, 28, 28, 1)`. Por conveniência, podemos passar o método [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy.reshape) um `-1` para qualquer dimensão que desejamos permanece o mesmo, portanto:

In [ ]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [ ]:
x_train.shape

In [ ]:
x_valid.shape

In [ ]:
x_train.shape, x_valid.shape

## Criando um Modelo Convolucional

Atualmente, muitos cientistas de dados iniciam seus projetos pegando emprestado as propriedades do modelo de um projeto semelhante. Supondo que o problema não seja totalmente único, há uma grande chance de que as pessoas tenham criado modelos com bom desempenho, publicados em repositórios on-line como [TensorFlow Hub](https://www.tensorflow.org/hub) e o [Catálogo NGC ](https://ngc.nvidia.com/catalog/models). Hoje, forneceremos um modelo que funcionará bem para esse problema.

<img src="../images/cnn.png" width=180 />

Já estudamos muitos dos diferentes tipos de camadas na palestra, e vamos passar por todos eles aqui com links para suas documentações. Em caso de dúvida, leia a documentação oficial (ou pergunte [stackoverflow](https://stackoverflow.com/)).

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))

### [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

<img src="../images/conv2d.png" width=400 />

Estas são as nossas camadas convolucionais 2D. Os kernels pequenos irão examinar a imagem de entrada e detectar recursos que são importantes para a classificação. Convoluções anteriores no modelo detectarão recursos simples, como linhas. Convoluções posteriores detectarão recursos mais complexos. Vejamos nossa primeira camada Conv2D:
```Píton
model.add(Conv2D(75 , (3,3) , passos = 1 , padding = 'mesmo'...)
```
75 refere-se ao número de filtros que serão aprendidos. (3,3) refere-se ao tamanho desses filtros. Os passos referem-se ao tamanho do passo que o filtro levará ao passar pela imagem. Preenchimento refere-se a se a imagem de saída criada a partir do filtro corresponderá ao tamanho da imagem de entrada.

### [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization)

Assim como normalizar nossas entradas, a normalização em lote dimensiona os valores nas camadas ocultas para melhorar o treinamento. [Leia mais sobre isso em detalhes aqui](https://blog.paperspace.com/busting-the-myths-about-batch-normalization/).

### [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)

<img src="../images/maxpool2d.png" width=400 />

O pooling máximo pega uma imagem e basicamente a reduz para uma resolução mais baixa. Ele faz isso para ajudar o modelo a ser robusto à translação (objetos se movendo de um lado para o outro) e também torna nosso modelo mais rápido.

### [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)

<img src="../images/dropout.png" width=400 />

Dropout é uma técnica para prevenir overfitting. Dropout seleciona aleatoriamente um subconjunto de neurônios e os desliga, para que eles não participem da propagação para frente ou para trás naquele passe específico. Isso ajuda a garantir que a rede seja robusta e redundante e não dependa de nenhuma área para obter respostas.

### [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)

Flatten pega a saída de uma camada que é multidimensional e a achata em uma matriz unidimensional. A saída é chamada de vetor de características e será conectada à camada de classificação final.

### [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)

Já vimos camadas densas antes em nossos modelos anteriores. Nossa primeira camada densa (512 unidades) toma o vetor de recursos como entrada e aprende quais recursos contribuirão para uma classificação específica. A segunda camada densa (24 unidades) é a camada de classificação final que gera nossa previsão.

## Resumindo o Modelo

Isso pode parecer muita informação, mas não se preocupe. Não é crítico entender tudo agora para treinar efetivamente os modelos convolucionais. Mais importante, sabemos que eles podem ajudar na extração de informações úteis de imagens e podem ser usados em tarefas de classificação.

Aqui, resumimos o modelo que acabamos de criar. Observe como ele tem menos parâmetros treináveis do que o modelo do notebook anterior:

In [ ]:
model.summary()

## Compilando o Modelo

Vamos compilar o modelo como antes:

In [ ]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

## Treinando o Modelo

Apesar da arquitetura do modelo muito diferente, o treinamento parece exatamente o mesmo. Execute a célula abaixo para treinar por 20 epochs e vamos ver se a precisão melhora:

In [ ]:
model.fit(x_train, y_train, epochs=10, verbose=1, validation_data=(x_valid, y_valid))

## Discussão dos Resultados

Parece que este modelo foi significativamente melhorado! A precisão do treinamento é muito alta e a precisão da validação também melhorou. Este é um ótimo resultado, pois tudo o que tivemos que fazer foi trocar por um novo modelo.

Você deve ter notado a precisão da validação saltando. Isso é uma indicação de que nosso modelo ainda não está generalizando perfeitamente. Felizmente, há mais que podemos fazer. Vamos falar sobre isso na próxima palestra.

## Resumo

Nesta seção, utilizamos vários novos tipos de camadas para implementar uma CNN, que teve um desempenho melhor do que o modelo mais simples usado na última seção. Esperamos que o processo geral de criação e treinamento de um modelo com dados preparados esteja começando a se tornar ainda mais familiar.

### Resetar o ambiente
Antes de prosseguir, execute a célula a seguir para resetar o ambiente. Isso é necessário para passar para o próximo notebook.

In [ ]:
import os
os._exit(00)

## Next

Nas últimas seções, você se concentrou na criação e no treinamento de modelos. Para melhorar ainda mais o desempenho, agora você voltará sua atenção para o *aumento de dados*, uma coleção de técnicas que permitirá que seus modelos treinem com mais e melhores dados do que os que você poderia ter originalmente à sua disposição.

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>